# 3.1 Parabolic model problem
We are solving the unsteady heat equation 

$$\text{find } u:[0,T] \to H_{0,D}^1 \quad \int_{\Omega} \partial_t u v + \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v = \int f v  \quad \forall v \in H_{0,D}^1, \quad u(t=0) = u_0$$

with a suitable advective field $b$ (the wind).

In [1]:
from ngsolve import *
from math import pi
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

* Geometry: $(-1,1)^2$
* Dirichlet boundaries everywhere
* Mesh

In [2]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), 
                 bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
Draw(mesh)
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")

u,v = fes.TnT()

time = 0.0
dt = 0.001

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

We define the field $b$ (the wind) as 

$$b(x,y) := (2y(1-x^2),-2x(1-y^2)).$$

In [3]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")
from ngsolve.internal import visoptions
visoptions.scalfunction = "wind:0"

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

* bilinear forms for 
 * convection-diffusion stiffness and 
 * mass matrix seperately.
* non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [4]:
a = BilinearForm(fes, symmetric=False)
a += 0.01*grad(u)*grad(v)*dx + b*grad(u)*v*dx
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += u*v*dx
m.Assemble()

## Implicit Euler method
$$
  \underbrace{(M + \Delta t A)}_{M^\ast} u^{n+1} = M u^n + \Delta tf^{n+1}
$$

or in an incremental form:

$$
  M^\ast (u^{n+1} - u^n) = - \Delta t A u^n + \Delta tf^{n+1}.
$$

* Incremental form: $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).
* For the time stepping method: set up linear combinations of matrices.
* (Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [5]:
mstar = m.mat.CreateMatrix()
print(m.mat.nze, a.mat.nze, mstar.nze)
print(mstar)


10798 10798 10798
Row 0:   0: 0.00520833   4: 0.00260417   31: 0.00260417   86: -0.000520833   87: 8.68056e-05   88: -0.000520833   89: 8.68056e-05   106: -0.000260417   107: -1.38913e-19   532: 0.000173611
Row 1:   1: 0.00520833   10: 0.00260417   11: 0.00260417   90: -0.000520833   91: 8.68056e-05   92: -0.000520833   93: 8.68056e-05   140: -0.000260417   141: 1.38913e-19   544: 0.000173611
Row 2:   2: 0.00520833   17: 0.00260417   18: 0.00260417   94: -0.000520833   95: 8.68056e-05   96: -0.000520833   97: 8.68056e-05   180: -0.000260417   181: 1.38913e-19   559: 0.000173611
Row 3:   3: 0.00825547   24: 0.00208821   25: 0.00203952   49: 0.00412774   98: -0.000417643   99: 6.96071e-05   100: -0.000407904   101: 6.7984e-05   102: -0.000825547   103: 0.000137591   222: -0.000208821   223: 1.05032e-19   226: -0.000203952   227: -1.01644e-19   571: 0.000139214   573: 0.000135968
Row 4:   0: 0.00260417   4: 0.017946   5: 0.00267639   31: 0.00629663   32: 0.00636885   86: -0.000520833   87

To access the entries we use the vector of nonzero-entries:

In [6]:
print(mstar.nze)
print(len(mstar.AsVector()))

10798
10798


In [7]:
print(mstar.AsVector())

 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 -1.38913e-19
 0.000173611
 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 1.38913e-19
 0.000173611
 0.00520833
 0.00260417
 0.00260417
 -0.000520833
 8.68056e-05
 -0.000520833
 8.68056e-05
 -0.000260417
 1.38913e-19
 0.000173611
 0.00825547
 0.00208821
 0.00203952
 0.00412774
 -0.000417643
 6.96071e-05
 -0.000407904
 6.7984e-05
 -0.000825547
 0.000137591
 -0.000208821
 1.05032e-19
 -0.000203952
 -1.01644e-19
 0.000139214
 0.000135968
 0.00260417
 0.017946
 0.00267639
 0.00629663
 0.00636885
 -0.000520833
 -8.68056e-05
 -0.000260417
 -1.40607e-19
 -0.000535278
 8.9213e-05
 -0.00125933
 0.000209888
 -0.00127377
 0.000212295
 -0.000267639
 -1.35525e-19
 -0.000369246
 1.96512e-19
 0.000173611
 0.000178426
 0.000246164
 0.00267639
 0.0163377
 0.00258899
 0.00557985
 0.00549245
 -0.000535278
 -8.9213e-05
 -0.000267639
 -1.4569e-19
 

Using the vector we can build the linear combination of the a and the m matrix:

In [8]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [22]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += gaussp*v*dx
f.Assemble()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

and the initial data: $u_0 = (1-y^2)x$

In [10]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
# Draw(gfu,mesh,"u")
print(gfu.vec)

 1.94289e-16
       0
 -1.94289e-16
 -3.33067e-16
 2.86808e-16
 -2.63678e-16
 -7.86408e-17
 -7.37257e-18
 -1.50343e-17
 1.27213e-16
 1.41669e-16
  0.4375
    0.75
  0.9375
       1
  0.9375
    0.75
  0.4375
 3.43475e-16
 -2.26671e-16
 3.23815e-16
 -1.50343e-17
 -6.4763e-17
 -1.17961e-16
 -2.8782e-16
 -0.4375
   -0.75
 -0.9375
      -1
 -0.9375
   -0.75
 -0.4375
 -0.292204
 -0.163794
 -0.0508035
 0.0511254
 0.134518
 0.237468
 0.677114
 0.783005
 0.762264
 0.649508
 0.440205
 0.217844
 0.160366
 0.0596285
 -0.0464766
 -0.132656
 -0.197695
 -0.290117
 -0.554216
 -0.714092
 -0.785823
 -0.770963
 -0.603682
 -0.170796
 -0.3661
 0.18394
 0.299895
 0.490307
 0.574829
 0.567809
 0.499239
 0.383151
 0.00455953
 0.228278
 -0.168782
 -0.300261
 -0.387316
 -0.565735
 -0.500955
 -0.571356
 -0.542583
 0.0199888
 0.410775
 0.370429
 -0.351091
 -0.153526
 0.0783996
 0.323985
 -0.322696
 -0.356898
 0.182727
 -0.0535613
 -0.11682
 0.134225
 3.83287e-15
 1.74706e-15
   0.125
 -5.41234e-15
 -2.66844e-15


In [11]:
res = gfu.vec.CreateVector()
tstep = 1 # time that we want to step over within one block-run

In [12]:
%%time
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 1.0000000000000007
CPU times: user 185 ms, sys: 1.88 ms, total: 186 ms
Wall time: 186 ms


### Alternative version with iterative solvers

* For a factorization of $M^\ast$ (${M^\ast}^{-1}$) we require a sparse matrix $M^\ast$ 
* To store $M^\ast$ as a sparse matrix requires new storage (and same memory layout of $A$ and $M$)
* For iterative solvers we only require the matrix (and preconditioner) applications
* `mstar = m.mat + dt * a.mat` has no storage but matrix-vector multiplications

iterative solver version (with Jacobi preconditining):

In [13]:
mstar_alt = m.mat + dt * a.mat
premstar_alt = m.mat.CreateSmoother() # + dt * a.mat.CreateSmoother()
from ngsolve.krylovspace import CGSolver
invmstar_alt = CGSolver(mstar_alt,premstar_alt, printrates='\r',\
                        tol=1e-8, maxiter=200)

print(premstar_alt)

Print base-matrix



In [14]:
%%time
tstep = 0.1
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar_alt * res
    t_intermediate += dt
    # print("\r t = {:24}, iteration steps: {}".format(time+t_intermediate,invmstar_alt.GetSteps()), end="")
    Redraw(blocking=False)
print("")
time+=t_intermediate

CG converged in 120 iterations to residual 2.4000569611388726e-11
CG converged in 120 iterations to residual 2.3859946035486514e-11
CG converged in 119 iterations to residual 2.971143362570477e-11
CG converged in 119 iterations to residual 2.941319768953749e-11
CG converged in 119 iterations to residual 2.912760615677311e-11
CG converged in 119 iterations to residual 2.884923667211385e-11
CG converged in 119 iterations to residual 2.857509363415097e-11
CG converged in 119 iterations to residual 2.83035473085238e-11
CG converged in 119 iterations to residual 2.8033735516861292e-11
CG converged in 119 iterations to residual 2.776524352058083e-11
CG converged in 119 iterations to residual 2.7497939216567172e-11
CG converged in 119 iterations to residual 2.7231890523197574e-11
CG converged in 119 iterations to residual 2.696732303400769e-11
CG converged in 119 iterations to residual 2.6704594181662315e-11
CG converged in 119 iterations to residual 2.6444170722411152e-11
CG converged in 119

CG converged in 117 iterations to residual 2.8018030120484547e-11
CG converged in 117 iterations to residual 2.7672075601398258e-11
CG converged in 117 iterations to residual 2.7330494076253446e-11
CG converged in 117 iterations to residual 2.699271727041542e-11
CG converged in 117 iterations to residual 2.665851796083035e-11
CG converged in 117 iterations to residual 2.6327962019919996e-11
CG converged in 116 iterations to residual 2.9070778176690693e-11
CG converged in 116 iterations to residual 2.8595474681692666e-11
CG converged in 116 iterations to residual 2.8141313150416782e-11
CG converged in 116 iterations to residual 2.7706181508134426e-11
CG converged in 116 iterations to residual 2.7288019971633942e-11
CG converged in 116 iterations to residual 2.688488774858913e-11
CG converged in 116 iterations to residual 2.649501510880811e-11
CG converged in 116 iterations to residual 2.6116840330168344e-11
CG converged in 116 iterations to residual 2.5749031677851262e-11
CG converged i

CG converged in 114 iterations to residual 2.527675047830448e-11
CG converged in 113 iterations to residual 2.877117387690804e-11
CG converged in 113 iterations to residual 2.8354820128479555e-11
CG converged in 113 iterations to residual 2.7947130553878318e-11
CG converged in 113 iterations to residual 2.7547829413878176e-11
CG converged in 113 iterations to residual 2.7156702389798765e-11
CG converged in 113 iterations to residual 2.677358196476341e-11
CG converged in 113 iterations to residual 2.639833419723286e-11
CG converged in 113 iterations to residual 2.6030847127232835e-11
CG converged in 113 iterations to residual 2.567102093960772e-11
CG converged in 113 iterations to residual 2.5318759911282696e-11
CG converged in 113 iterations to residual 2.4973966092383764e-11
CG converged in 113 iterations to residual 2.4636534613776352e-11
CG converged in 113 iterations to residual 2.4306350474512004e-11
CG converged in 113 iterations to residual 2.3983286639502616e-11
CG converged in

## Supplementary material:
* [Time dependent r.h.s.](#TDRHS)
* [Time dependent boundary data](#TDBND)
* [Runge-Kutta time integration](#RK)
* [VTK Output](#VTK)

## Supplementary 1: time-dependent r.h.s. data <a class="anchor" id="TDRHS"></a>

Next: time-dependent r.h.s. data $f=f(t)$:

* Use `Parameter` t representing the time. 
* A `Parameter` is a constant `CoefficientFunction` the value of which can be changed with the `Set`-function.

In [15]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [17]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
scene = Draw(gausspt,mesh,order=3)
time = 0.0
from time import sleep
while time < 10 - 0.5 * dt:
    t.Set(time)
    scene.Redraw()
    time += 1e-3
    sleep(1e-3)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

Accordingly we define a different linear form which then has to be assembled in every time step.

In [ ]:
ft = LinearForm(fes)
ft += gausspt*v*dx
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
scene = Draw(gfu,mesh,"u")

In [ ]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print("")
time+=t_intermediate

## Supplementary 2: Time dependent boundary conditions <a class="anchor" id="TDBND"></a>


* $u|_{\partial \Omega} = u_D(t)$, $f=0$
* implicit Euler time stepping method, non-incremental form:

  $$
    M^\ast u^{n+1} = (M + \Delta t A) u^{n+1} = M u^n
  $$  
  
* Homogenize w.r.t. to boundary conditions, i.e. we split 

  $$ u^{n+1} = u^{n+1}_0 + u^{n+1}_D $$ 
  
  where $u^{n+1}_D$ is a (discrete) function with correct boundary condition:  
  
$$
  {M^\ast} u^{n+1}_0 = M u^n - {M^\ast} u^{n+1}_D
$$


In [ ]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
scene = Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

In [ ]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print("")
time+=t_intermediate

## Supplementary 3: Singly diagonally implicit Runge-Kutta methods <a class="anchor" id="RK"></a>


We consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\frac{d}{dt} u = M^{-1} F(u) = -M^{-1} \cdot A u$ are of the form:

$$
  u^{n+1} = u^n + \Delta t M^{-1} \sum_{i=0}^{s-1} b_i k_i
$$

with

$$
k_i = - A u_i \quad \text{ where $u_i$ is the solution to } \quad 
(M + a_{ii} \Delta t A) u_i = M u^n - \Delta t \sum_{i=0}^{i-1} a_{ij} k_j, \quad i=0,..,s-1.
$$

The coefficients a,b and c are stored in the butcher tableau:

$$
\begin{array}{c|cccc}
c_0 & a_{00} & 0 & \ddots& \\
c_1 & a_{10} & a_{11} & 0 & \ddots \\
\vdots & \vdots & \vdots & \ddots & 0\\
c_{s-1} & a_{s-1,0} & a_{s-1,1} & \dots& a_{s-1,s-1} \\ \hline
 & b_{0} & b_1 & \dots& b_{s-1} \\
\end{array}
$$

For an SDIRK method we have $a^\ast = a_{ii},~i=0,..,s-1$.

Simplest example: Implicit Euler
$$
\begin{array}{c|c}
1 & 1 \\ \hline
 & 1  \\
\end{array}
$$

In [ ]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

We can use for example the 2 stage SDIRK (order 3) method

$$
\begin{array}{c|cc}
p & p & 0 \\
1-p & 1-2p & p \\ \hline
 & 1/2 & 1/2 \\
\end{array}
$$

with $p = \frac{3 - \sqrt{3}}{6}$.

In [ ]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p
    

We can use for example the 5 stage SDIRK (order 4) method
$$
\begin{array}{c|cccc}
1/4 & 1/4  \\
3/4 & 1/2 & 1/4 & \\
11/20 & 17/50 & -1/25 & 1/4 \\
1/2 & 371/1360 & -137/2720 & 15/544 & 1/4 \\
1 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\ \hline
 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\
\end{array}
$$

In [ ]:
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

### SDIRK2 :

In [ ]:
butchertab = sdirk2()   
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

We have to update the $M^\ast$ matrix and reset initial data

In [ ]:
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
scene = Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

In [ ]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    Mu0.data = m.mat * gfu.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(uD,BND)
        ui.data = gfu.vec; rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        # compute k[i] from ui
        k[i].data = - a.mat * ui
    t_intermediate += dt; t.Set(time+t_intermediate)
    # Adding up the ks:
    gfu.Set(uD,BND)
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0        
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
print(""); time+=t_intermediate            

## Supplementary 4: VTK Output ("exporting the nice pictures") <a class="anchor" id="VTK"></a>

* see also https://ngsolve.org/blog/ngsolve/2-vtk-output
* or `py_tutorials/vtkout.py` (ngsolve repository)

Outputting the nice pictures to vtk (to visualize with paraview):

In [ ]:
vtk = VTKOutput(mesh,coefs=[gfu],
                names=["sol"],
                filename="vtk_example1",
                subdivision=3)
vtk.Do()

In [ ]:
%%bash
ls vtk_example1.*

You can also export vector fields:

In [ ]:
vtk = VTKOutput(mesh,coefs=[gfu,grad(gfu)],names=["sol","gradsol"],filename="vtk_example2",subdivision=3)
vtk.Do()

In [ ]:
#%%bash
#paraview vtk_example2.vtk

And time dependent data:

In [ ]:
vtk = VTKOutput(mesh,coefs=[gfu],names=["sol"],filename="vtk_example3",subdivision=3)
gfu.Set((1-y*y)*x)
vtk.Do(time=0)
time = 0
tstep = 1 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
i = 0
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    scene.Redraw()
    i += 1
    if (i%10 == 0):
        vtk.Do(time=t_intermediate)
print("")

Call paraview on gerenated data (if installed):

In [ ]:
%%bash
if ! command -v paraview &> /dev/null
then
    echo "paraview is not installed."
    exit
else
    paraview vtk_example3.pvd
fi